In [ ]:
# tests to find solution for https://github.com/c-scale-community/use-case-hisea/issues/28

from datetime import timedelta, datetime
from pathlib import Path
import subprocess
import xarray as xr # note dependencies: dask, netCDF4

vars = ('thetao', 'bottomT', 'so', 'zos', 'uo', 'vo')
date_min = '2022-04-01'
date_max = '2022-04-05'

delta = datetime.strptime(date_max, '%Y-%m-%d') - datetime.strptime(date_min, '%Y-%m-%d')

var = vars[0]
i = 0
day = datetime.strptime(date_min, '%Y-%m-%d').date() + timedelta(days=i)
check_file = Path('/Users/backeb/Documents/data/tmp/cmems_'+str(var)+'_'+str(day)+'.nc')

j=0
while not check_file.is_file():
    print('j '+str(i))
    print(check_file.is_file())
    open(check_file, 'a')
    j+=1

print(check_file.is_file())
print(j)
    

In [ ]:
#### IGNORE THIS, I WAS BEING A RETARD #### 
# works fine with xr.open_mfdataset('/Users/backeb/Documents/data/tmp/*no3*.nc')
# I was getting error in xr.open_mfdataset('/Users/backeb/Documents/data/tmp/*no3*.nc') BUT I WAS BEING AN IDIOT
# ValueError: Could not find any dimension coordinates to use to order the datasets for concatenation
# and using ds = xr.open_mfdataset('*no3*.nc', concat_dim='time', combine='nested') creates 25 time dimensions instead of the 5 expected

from pathlib import Path
import xarray as xr

datasets = []
#for path in Path('/Users/backeb/Documents/data/tmp/').iterdir():
for path in sorted(Path('/Users/backeb/Documents/data/tmp/').rglob('*no3*.nc')):
    print(path)
    ds = xr.open_dataset(path)
    datasets.append(ds)
combined = xr.concat(datasets, dim='time')

print(combined)

In [ ]:
# for download_era5.py get an error when iterating over months
# e.g. from 2021-07-01 to 2022-06-31 the monthstr is empty
# this is because I'm trying to do `for i in range(7, 6, 1): ...`
# tests to find a better way to iterate over a period
# TODO think about extracting one netcdf per day and concat after using xarray.open_mfdataset, i.e. as in cmems download approach

import pandas as pd
from datetime import datetime, timedelta
import numpy as np

date_min = '2021-07-01'
date_max = '2022-06-30'

yearstr = []
pdyears = pd.period_range(start=date_min, end=date_max, freq='Y')
[yearstr.append(f'{year:0>4}') for year in pdyears.year]
print(yearstr)

monthstr = []
pdmonths = pd.period_range(start=date_min, end=date_max, freq='M')
#print(np.unique(pdmonths.month))
[monthstr.append(f'{month:0>2}') for month in np.unique(pdmonths.month)]
print(monthstr)

daystr = []
pddays = pd.period_range(start=date_min, end=date_max, freq='D')
#print(np.unique(pddays.day))
[daystr.append(f'{day:0>2}') for day in np.unique(pddays.day)]
print(daystr)



In [ ]:
# tests to change download_era5.py to deal with CDS API limitation: https://github.com/c-scale-community/use-case-hisea/issues/30
from datetime import timedelta, datetime

date_min = '2021-07-01'
date_max = '2022-06-30'
vars = ('10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure', '2m_dewpoint_temperature', 'relative_humidity', 'surface_net_solar_radiation', '2m_temperature', 'total_cloud_cover')

vars = list(vars)
print(vars)

delta = datetime.strptime(date_max, '%Y-%m-%d') - datetime.strptime(date_min, '%Y-%m-%d')
i = 0
day = datetime.strptime(date_min, '%Y-%m-%d').date() + timedelta(days=i)
yearstr = [f'{day.year:0>4}']
print(yearstr)
monthstr = [f'{day.month:0>2}']
print(monthstr)
daystr = [f'{day.day:0>2}']
print(daystr)



In [17]:
# tests to figure out https://github.com/c-scale-community/use-case-hisea/issues/31
from datetime import timedelta, datetime
from pathlib import Path
import subprocess
import xarray as xr

username = 'insert'
password = 'insert'
longitude_min = 22.5
longitude_max = 24.5
latitude_min = 36.5
latitude_max = 38.5
date_min = '2022-04-01'
date_max = '2022-04-02'
vars = ('thetao', 'bottomT', 'so', 'zos', 'uo', 'vo')

#make the /data/tmp directory if it does not exist
Path('/data/tmp').mkdir(parents=True, exist_ok=True)
delta = datetime.strptime(date_max, '%Y-%m-%d') - datetime.strptime(date_min, '%Y-%m-%d')
for var in vars:
        for i in range(delta.days+1):
                max_runs = 2
                run = 0
                day = datetime.strptime(date_min, '%Y-%m-%d').date() + timedelta(days=i)
                check_file = Path('/data/tmp/cmems_'+str(var)+'_'+str(day)+'.nc')
                while not check_file.is_file() and run < max_runs:
                        try:
                                subprocess.run(['python', '-m', 'motuclient',
                                        '--motu', 'https://nrt.cmems-du.eu/motu-web/Motu',
                                        '--service-id', 'GLOBAL_ANALYSIS_FORECAST_PHY_001_024-TDS',
                                        '--product-id', 'global-analysis-forecast-phy-001-024',
                                        '--longitude-min',str(longitude_min),
                                        '--longitude-max', str(longitude_max),
                                        '--latitude-min', str(latitude_min),
                                        '--latitude-max', str(latitude_max),
                                        '--date-min', str(day)+' 12:00:00',
                                        '--date-max', str(day)+' 12:00:00',
                                        '--depth-min', '0.493',
                                        '--depth-max', '5727.918000000001',
                                        '--variable', str(var),
                                        '--out-dir', '/data/tmp',
                                        '--out-name', 'cmems_'+str(var)+'_'+str(day)+'.nc',
                                        '--user', username,
                                        '--pwd', password],
                                        check=True,
                                        timeout=300)
                        except subprocess.TimeoutExpired as e:
                                print(var)
                                print(e.stdout)
                                print(e.stderr)
                                continue
                        else:
                                break
                        finally:
                                run += 1
        ds = xr.open_mfdataset('/data/tmp/cmems_'+var+'_*.nc')
        ds.to_netcdf('/data/cmems_'+var+'.nc')

thetao
2022-08-03 11:28:09.034 [ INFO] Asynchronous mode set
2022-08-03 11:28:09.035 [ INFO] Authenticating user bbackeberg for service https://nrt.cmems-du.eu/motu-web/Motu
2022-08-03 11:28:09.918 [ INFO] Requesting file to download (this can take a while)...
2022-08-03 11:28:15.486 [ INFO] Authenticating user bbackeberg for service https://nrt.cmems-du.eu/motu-web/Motu
2022-08-03 11:28:21.788 [ INFO] The product is ready for download
2022-08-03 11:28:21.788 [ INFO] Downloading file (this can take a while)...
2022-08-03 11:28:21.953 [ INFO] File type: application/x-netcdf
2022-08-03 11:28:21.953 [ INFO] File size: 65.8 kB (65776 B)
2022-08-03 11:28:21.954 [ INFO] Downloading file /Users/backeb/Documents/data/tmp/cmems_thetao_2022-04-01.nc
2022-08-03 11:28:22.014 [ INFO] -  65.8 kB (99.6%)
2022-08-03 11:28:22.015 [ INFO] -  65.8 kB (100.0%)
2022-08-03 11:28:22.015 [ INFO] Processing  time : 0:00:12.919384
2022-08-03 11:28:22.015 [ INFO] Downloading time : 0:00:00.061288
2022-08-03 11:2